### Geography of Flickr Images Sentiment
This notebook does the following:
- Loads the image data from the landmark directory specified below
- Uses Microsoft Face API to get estimate emotion of faces
- Averages sentiment and emotions by Landmarks
- Visualize the results

In [1]:
pumaFile = '/home/urwa/Documents/Courses/Sem1/PUI/PUIData/Old Data/PUMA/geo_export_f0519f41-2898-4829-af71-3556b078f017.shp'
imageDirectory = 'processed/'

In [2]:
from __future__ import print_function
import os
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import requests 
import json
import numpy as np
import io
import json
import nltk
import shapely
from geopandas.tools import sjoin
from fiona.crs import from_epsg
import time

import requests
from scipy import stats

In [3]:
subscription_key = ''
assert subscription_key

In [4]:
face_api_url = 'https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect'

headers = { 'Content-Type': 'application/octet-stream','Ocp-Apim-Subscription-Key': subscription_key }
    
params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age,gender,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
}

In [5]:
def getSentiment(emo):
    return emo['happiness'] - (emo['anger'] + emo['contempt'] + emo['disgust'] + emo['fear'] + emo['sadness'])

In [6]:
def classifyEmotion(emo):
    emotions = emo.keys()
    scores = emo.values()
    return emotions[np.argmax(scores)]

In [7]:
def getDirectorySentiment(directory):
    images = os.listdir(directory)
    print(directory)
    print('images: ',len(images))
    sentList = []
    emoList = []
    for img in images:
        print(img)
        image_url = directory+'/'+img
        data = open(image_url, 'rb').read()
        response = requests.post(face_api_url, params=params, headers=headers, data=data )
        res = response.json()
        try :
            if len(res)>0:
                for r in res:
                    if 'emotion' in r['faceAttributes']:
                        sentList.append(getSentiment(r['faceAttributes']['emotion']))
                        emoList.append(classifyEmotion(r['faceAttributes']['emotion']))
        except Exception as e:
            print(e)
            print(res)
        time.sleep(3)
    print('Faces:',len(sentList))
    aggEmo = pd.Series(emoList).value_counts()
    resultDict = dict(zip(aggEmo.index, aggEmo.values))
    resultDict['sentiment'] = np.mean(sentList)  
    with open(directory+'/'+'sentimentData', 'w') as outfile:
        json.dump(resultDict, outfile)
    return resultDict

In [8]:
directories = os.listdir(imageDirectory)

In [9]:
pumaSentList = []
for dirr in directories:
    resultDict = getDirectorySentiment(imageDirectory+dirr)
    resultDict['puma'] = dirr
    pumaSentList.append(resultDict)

processed/brooklynBridge
images:  18
33769945048_f54ca58d9f_c.jpg
32716687607_bd913c3900_c.jpg
33810031558_b33e8c741f_c.jpg
46613464195_df8f58bbe6_c.jpg
47518736861_c9098f7324_c.jpg
47398173061_0ee2491b9f_c.jpg
46715430544_207dacf8b5_c.jpg
47134288572_4afec15471_c.jpg
47236155811_dd6fc6651d_c.jpg
40552715913_5207662d3b_c.jpg
32575320917_15b11ae325_c.jpg
sentimentData
string indices must be integers
{u'error': {u'message': u'Image size is too small.', u'code': u'InvalidImageSize'}}
46675027964_5641410f80_c.jpg
40722314083_6075443605_c.jpg
32501808067_deb0494043_c.jpg
32576287317_1dfb2962af_c.jpg
32574102077_53a05f7b92_c.jpg
33641844238_d018820c4a_c.jpg
Faces: 13
processed/flatIronBuilding
images:  37
46289404565_d3046252a5_c.jpg
47168965742_1317da0915_c.jpg
32610313977_d08b41ba9e_c.jpg
39986378403_27260e0b5e_c.jpg
46506489294_04e728aaf5_c.jpg
46565794625_4b1f030e7a_c.jpg
46977461262_15046fea0a_c.jpg
46036903105_2aa36c35b0_c.jpg
32610345247_bd8c437dfe_c.jpg
40238077603_b5c15d13ae_c.jpg
4

In [10]:
pumaSent = pd.DataFrame(pumaSentList)
pumaSent

,happiness,neutral,puma,sentiment,surprise
0,9,4,brooklynBridge,0.703923,NaN
1,12,20,flatIronBuilding,0.280029,2.0
2,25,41,timeSquare,0.335621,NaN
3,35,21,woldTradecenter,0.596930,1.0


In [11]:
pumaSent.to_csv('faceLandmarkSent.csv',index=False)